In [1]:
import pandas as pd
import json

# TODO :

* market basket analysis: pour determiner les corelations entre les evenements
* Variable Importance with ML Decision Tree: separer les donées en n classe pour variable interreser. n étant les velurs unique dans la classe et evaluer l'importance des valeurs

In [2]:
def add_cols(df: pd.DataFrame, cols: list, init_val):
    for col in cols:
        df[str(col)] = init_val
        
    return df


def prefix_with_col(df: pd.DataFrame) -> pd.DataFrame:
    result = add_cols(pd.DataFrame(), df.columns, '')
    result['Num_Acc'] = df['Num_Acc']
    for col in df.columns:
        if str(col) == 'Num_Acc': continue
        result[str(col)] = df[col].map(lambda x: f'{col}__{convert_to_int(x)}')
            
    return result

In [6]:
data_folder = '../data/accidents-in-france-from-2005-to-2016/'

caracteristics = pd.read_csv(data_folder + 'caracteristics.csv', encoding='latin-1')

/opt/miniconda3/envs/ml-skl/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
caracteristics = caracteristics.drop(columns=['adr', 'lat',  'long', 'gps'])

In [8]:
pd.unique(caracteristics['an'])

array([16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5])

In [9]:
users = pd.read_csv(data_folder + 'users.csv', encoding='latin-1')

In [10]:
users = users.drop(columns=['num_veh'])

In [11]:
users.head(3)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais
0,201600000001,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0
1,201600000001,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0
2,201600000002,1.0,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0


In [12]:
vehicles = pd.read_csv(data_folder + 'vehicles.csv', encoding='latin-1')

In [13]:
vehicles = vehicles.drop(columns=['num_veh', 'senc', 'occutc', 'obs', 'obsm', 'manv', 'choc'])

In [14]:
vehicles.head(3)

,Num_Acc,catv
0,201600000001,7
1,201600000001,2
2,201600000002,7


pour analyser plus finement les données et pour augmenter les nombre les patterns on peut faire certaines pretraitemnt. notament transformer les annnée de naissance en tranche d'age. par exemple tranche d'age entre 18-25, 26-35, 36-50, 51-*.

In [15]:
dataset = caracteristics.merge(users, on='Num_Acc')
dataset = dataset.merge(vehicles, on='Num_Acc')

In [16]:
dataset.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,catv
0,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,7
1,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,2
2,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,7
3,201600000001,16,2,1,1445,1,2,1,8.0,3.0,...,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,2
4,201600000002,16,3,16,1800,1,2,6,1.0,6.0,...,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0,7


In [17]:
caracteristics = None
users = None
vehicles = None

In [18]:
def convert_to_int(val):
    try:
        return int(val)
    except ValueError:
        return -1

In [19]:
age_ranges = {
    1: (0, 12),
    2: (13, 17),
    3: (18, 25),
    4: (26, 35),
    5: (36, 50),
    6: (51, 110)
}

In [20]:
def get_age_range(age_ranges, age):
    for a in age_ranges:
        if a[0] <= age <= a[1]:
            return age_ranges[a]

    raise ValueError(f'invalide age ranges {age}')

def user_age_range(dataset: pd.DataFrame, age_ranges: dict):
    
    ar = {v: k for k, v in age_ranges.items()}
    
    dataset['age_range'] = 0
    
    for i in dataset.index:
        an = convert_to_int(dataset.at[i, 'an'])
        an_nais = convert_to_int(dataset.at[i, 'an_nais'])
        if an == -1 or an_nais == -1: 
            continue
    
        age = (2000 + an) - an_nais
        dataset.at[i, "age_range"] = get_age_range(ar, age)


In [ ]:
user_age_range(dataset, age_ranges)

In [ ]:
dataset.head(3)

In [ ]:
f = open('frequent_info.json', 'r')
data_range = json.load(f)

def key_gen(col_name, val):
    return f'{col_name}__{val}'

def get_unique_id(unique_ids: dict, col_name: str, val):

    vi = convert_to_int(val)

    if col_name in data_range:
        r = data_range[col_name]['range']
        if not (r[0] <= vi <= r[1]):
            vi = -1

    k = key_gen(col_name, vi)

    if k not in unique_ids.keys():
        unique_ids[k] = len(unique_ids.keys())

    return unique_ids[k]


def transform_to_transaction(df: pd.DataFrame):
    unique_ids = {}

    result = add_cols(pd.DataFrame(), df.columns, -1)
    result['Num_Acc'] = df['Num_Acc']

    for col in df.columns:
        if str(col) == 'Num_Acc': continue

        result[str(col)] = df[col].map(lambda x: get_unique_id(unique_ids, str(col), x))

    return result, unique_ids

In [125]:
dataset_sub = dataset.iloc[:50000]

In [126]:
dataset_tr, unique_ids = transform_to_transaction(dataset_sub)

In [127]:
len(unique_ids.keys())

1519

In [90]:
dataset_tr.head(3)

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,...,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,catv,age_range
0,201600000001,0,1,13,44,495,500,502,511,520,...,1371,1375,1377,1385,1404,1414,1423,1428,1529,1553
1,201600000001,0,1,13,44,495,500,502,511,520,...,1371,1375,1377,1385,1404,1414,1423,1428,1530,1553
2,201600000001,0,1,13,44,495,500,502,511,520,...,1372,1376,1378,1386,1404,1414,1423,1429,1529,1554


In [91]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()

In [92]:
dataset_values = dataset_tr.values

In [93]:
dataset_values = dataset_values[:, 1:] # remove Num_Acc

In [94]:
sub_sample_dataset = dataset_values[:50000]

In [95]:
sub_sample_dataset[:2]

array([[   0,    1,   13,   44,  495,  500,  502,  511,  520,  527, 1264,
        1357, 1367, 1371, 1375, 1377, 1385, 1404, 1414, 1423, 1428, 1529,
        1553],
       [   0,    1,   13,   44,  495,  500,  502,  511,  520,  527, 1264,
        1357, 1367, 1371, 1375, 1377, 1385, 1404, 1414, 1423, 1428, 1530,
        1553]])

In [96]:
te_ary = te.fit(sub_sample_dataset).transform(sub_sample_dataset)

In [97]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [98]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [99]:
import json

def filter_binary_dataset(binary_df: pd.DataFrame, unique_ids:dict, original_df: pd.DataFrame):
    
    f = open('frequent_info.json', 'r')
    json_data = json.load(f)
    
    to_keep = []
    
    for col_name in json_data:
        col = json_data[col_name]
        
        for i in range(col['range'][0], col['range'][1]+1, 1):
            k = key_gen(col_name, i)
            if k in unique_ids:
                 to_keep.append(unique_ids[k])
    
    unknown_col = set(dataset_sub.columns) - set(json_data.keys())
    
    to_remove = []
    for col in unknown_col:
        if col == 'Num_Acc': continue
        
        unique_vals = pd.unique(original_df[col])
        for u_val in unique_vals:
            k = key_gen(col, u_val)
            if k in unique_ids:
                to_keep.append(unique_ids[k])
    
    to_keep = sorted(to_keep)
    
    return binary_df[to_keep]

In [100]:
df = filter_binary_dataset(df, unique_ids, dataset_sub)

In [103]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1535,1543,1550,1553,1554,1555,1556,1557,1558,1559
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [104]:
from mlxtend.frequent_patterns import association_rules, fpmax, fpgrowth, apriori

In [115]:
frequent_itemsets = fpmax(df, min_support=0.3, use_colnames=True)
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

# association_rules(frequent_itemsets, min_threshold=0.9)
#frequent_itemsets

In [107]:
id_inv_map = {v: k for k, v in unique_ids.items()}

In [108]:
def decompose_key(k:str):
    return k.split('__')[0], int(k.split('__')[1])

In [109]:
# save resulte

frequent_itemsets.to_csv('closed_itemset.csv', index=False)

with open('id_inv_map.json', 'w') as f:
    json.dump(id_inv_map, f)


In [116]:
f = open('descritption.json', 'r')
desc = json.load(f)

res = []
for i, itemset in frequent_itemsets.iterrows():
    l = [itemset['support']]
    
    it = []
    for i in itemset['itemsets']:
        col_name , val = decompose_key(id_inv_map[i])
        d = '___'
        col_desc = desc[col_name]['description']
        
        if col_desc['type'] == 'mono':
            d = col_desc['value']
        else:
            d = col_desc[str(val)]
        
        it.append((col_name, val, d))
        
    res.append((it, l))


In [117]:
s = 0
for r in res: #[s:s + 10]:
    print(r)
    print()

([('an', 16, "Anne de l'accident"), ('agg', 2, 'En agglomération')], [0.31974])

([('an', 16, "Anne de l'accident"), ('age_range', 6, '51 - 110'), ('actp', 0, 'non spécifié ou non applicable')], [0.30946])

([('an', 16, "Anne de l'accident"), ('actp', 0, 'non spécifié ou non applicable'), ('sexe', 2, 'Femme')], [0.31596])

([('an', 16, "Anne de l'accident"), ('grav', 3, 'Blessés hospitalisés'), ('actp', 0, 'non spécifié ou non applicable')], [0.3198])

([('an', 16, "Anne de l'accident"), ('actp', 0, 'non spécifié ou non applicable'), ('grav', 1, 'Indemne'), ('int', 1, 'Hors intersection')], [0.30136])

([('an', 16, "Anne de l'accident"), ('trajet', 5, 'Promenade - loisirs'), ('sexe', 1, 'Homme')], [0.30052])

([('an', 16, "Anne de l'accident"), ('trajet', 5, 'Promenade - loisirs'), ('lum', 1, 'Plein Jour')], [0.30346])

([('an', 16, "Anne de l'accident"), ('actp', 0, 'non spécifié ou non applicable'), ('trajet', 5, 'Promenade - loisirs'), ('int', 1, 'Hors intersection')], [0.31228])

(